# Topic modelling on news data 25 topics

- Data is taken from kaggle.
- Applying the topic modeling the see the results for 25 topics
- And choose the results for 2^nd level topic modeling

In [ ]:
import pandas as pd
data=pd.read_csv('abcnews-date-text.csv',error_bad_lines=False)

In [ ]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [ ]:
print(len(documents))
print(documents[:5])

# Data preprocessing 
## Different process we do here like as Tokenization ,lemetization and stemming the data
- we wanted to convert the data to their normal form for example 'stolen' would converted to steal
### Here nltk is used for removing different language rather than english (Hindi and urdu like that word would be removed)
- In preprocessing actually we remove all the punctuation marks , exclamatory marks and commas 

In [ ]:

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
               
            

# Lemmatization 
## Lemmatization is used for gouping of word that's contains the same meaning(synonyms,antonyms)
# Tokenization
## Tokenization is used for keeps the word having meaningfull meaning
- This is used for removal of word like if,the ,a,an that word doesn't make any sense in Topic
# Stemming
## Stemming is used for convert the word into their root form

In [ ]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

# preview data after preprocessing 
- How the data will look like 

In [ ]:
# Select a document to preview after preprocessing
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))



In [ ]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

### Dictionary is formed for 

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

## Filtering the word

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

In [ ]:
import pickle
pickle.dump(bow_corpus, open('bow_corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=25, id2word=dictionary, passes=2, workers=2)
lda_model.save('model25.gensim')

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
processed_docs[4310]

In [ ]:
unseen_document = 'How a Pentagon de to help him identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('bow_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model25.gensim')


In [ ]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

## As topic related to health is syncronised well so we increase the number of topic stop here.
